In [1]:
import json
import numpy as np
import pandas as pd
from dateutil import parser
import preprocess_hop_log, merge_hop_tick, aggregate_rows_by_time
import os

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

In [2]:
# path_hoplog = '../raw_data/hopLogger/12/'
# path_ticklog = '../raw_data/tickLogger/12/'
# path_mergeddf = '../preprocessed_data/12/'

DIR_NAME = 108

path_hoplog = '../raw_data/hopLogger/' + str(DIR_NAME) + '/'
path_ticklog = '../raw_data/tickLogger/' + str(DIR_NAME) + '/'
# path_hoplog = '../../wowsan/log/hopLogger/' + str(DIR_NAME) + '/'
# path_ticklog = '../../wowsan/log/tickLogger/' + str(DIR_NAME) + '/'
path_mergeddf = '../preprocessed_data_sttransformer/' + str(DIR_NAME) + '/'


In [3]:
NUM_BROKERS = 5
BROKER_PORT = 50003

## Hop Log 전처리

In [4]:
# json 파일을 열고 각 줄을 개별적으로 파싱
# path = '../raw_data/hopLogger/116/'
data = []
with open(path_hoplog + f'{BROKER_PORT}.json', 'r') as file:
    for line in file:
        data.append(json.loads(line))

# 데이터를 DataFrame으로 변환
hop_df_raw = pd.DataFrame(data)
type(hop_df_raw)

pandas.core.frame.DataFrame

In [5]:
hop_df_raw

,Node,PerformanceInfo,level,msg,time
0,localhost:50003,"[{'BrokerId': 'localhost:50004', 'Cpu': '99.642704', 'Memory': '23363584', 'QueueLength': '0', 'QueueTime': '0.307559', 'AverageQueueTime': '', 'ServiceTime': '0.000000', 'AverageServiceTime': '', 'ResponseTime': '0.307559', 'InterArrivalTime': '0.000000', 'Throughput': '3.251413', 'AverageThroughput': '', 'Timestamp': '2025-01-05 12:13:45.4207739 +0900 KST m=+4.463063101'}, {'BrokerId': 'localhost:50003', 'Cpu': '99.809589', 'Memory': '23396352', 'QueueLength': '0', 'QueueTime': '0.104737', 'AverageQueueTime': '0.104737', 'ServiceTime': '0.000000', 'AverageServiceTime': '0.000000', 'ResponseTime': '0.104737', 'InterArrivalTime': '0.000000', 'Throughput': '9.547724', 'AverageThroughput': '9.547724', 'Timestamp': '2025-01-05 12:13:45.5275137 +0900 KST m=+4.580801801'}]",info,Advertisement(apple > 0.25403618120212357) a0938dd9-1e2c-4b30-af19-1da2b3c535b0,2025-01-05 12:13:45
1,localhost:50003,"[{'BrokerId': 'localhost:50004', 'Cpu': '99.646627', 'Memory': '23891968', 'QueueLength': '0', 'QueueTime': '0.301959', 'AverageQueueTime': '', 'ServiceTime': '0.314640', 'AverageServiceTime': '', 'ResponseTime': '0.616599', 'InterArrivalTime': '0.681290', 'Throughput': '1.621800', 'AverageThroughput': '', 'Timestamp': '2025-01-05 12:13:47.4590507 +0900 KST m=+6.501333201'}, {'BrokerId': 'localhost:50003', 'Cpu': '100.007887', 'Memory': '23699456', 'QueueLength': '0', 'QueueTime': '0.101049', 'AverageQueueTime': '0.102893', 'ServiceTime': '0.107739', 'AverageServiceTime': '0.107739', 'ResponseTime': '0.208788', 'InterArrivalTime': '1.020634', 'Throughput': '4.789538', 'AverageThroughput': '4.747612', 'Timestamp': '2025-01-05 12:13:47.5651013 +0900 KST m=+6.618382701'}]",info,Advertisement(apple > 0.832939761751119) 1d01f9f5-6c5a-4d21-a86b-2c50c72fd32e,2025-01-05 12:13:47
2,localhost:50003,"[{'BrokerId': 'localhost:50004', 'Cpu': '99.685872', 'Memory': '24465408', 'QueueLength': '1', 'QueueTime': '0.300817', 'AverageQueueTime': '', 'ServiceTime': '0.301745', 'AverageServiceTime': '', 'ResponseTime': '0.602562', 'InterArrivalTime': '0.608465', 'Throughput': '1.659579', 'AverageThroughput': '', 'Timestamp': '2025-01-05 12:13:48.8985125 +0900 KST m=+7.940790301'}, {'BrokerId': 'localhost:50003', 'Cpu': '100.003245', 'Memory': '24051712', 'QueueLength': '0', 'QueueTime': '0.114638', 'AverageQueueTime': '0.106808', 'ServiceTime': '0.106049', 'AverageServiceTime': '0.106894', 'ResponseTime': '0.220688', 'InterArrivalTime': '1.160354', 'Throughput': '4.531290', 'AverageThroughput': '4.679404', 'Timestamp': '2025-01-05 12:13:49.0184874 +0900 KST m=+8.071764001'}]",info,Advertisement(apple > 0.3954138949680233) 85c0e4da-8772-4c76-b050-6ff996377687,2025-01-05 12:13:49
3,localhost:50003,"[{'BrokerId': 'localhost:50004', 'Cpu': '99.843099', 'Memory': '25096192', 'QueueLength': '2', 'QueueTime': '0.450594', 'AverageQueueTime': '', 'ServiceTime': '0.307151', 'AverageServiceTime': '', 'ResponseTime': '0.757744', 'InterArrivalTime': '0.473828', 'Throughput': '1.319706', 'AverageThroughput': '', 'Timestamp': '2025-01-05 12:13:49.2146142 +0900 KST m=+8.256890901'}, {'BrokerId': 'localhost:50003', 'Cpu': '99.942109', 'Memory': '24301568', 'QueueLength': '1', 'QueueTime': '0.101826', 'AverageQueueTime': '0.105563', 'ServiceTime': '0.118639', 'AverageServiceTime': '0.110809', 'ResponseTime': '0.220466', 'InterArrivalTime': '0.762020', 'Throughput': '4.535857', 'AverageThroughput': '4.621671', 'Timestamp': '2025-01-05 12:13:49.3204852 +0900 KST m=+8.373760801'}]",info,Advertisement(apple > 0.40280118482410926) 14bbdb96-e0d9-4e1a-b875-91a242abdb72,2025-01-05 12:13:49
4,localhost:50003,"[{'BrokerId': 'localhost:50003', 'Cpu': '100.544437', 'Memory': '24399872', 'QueueLength': '0', 'QueueTime': '0.208244', 'AverageQueueTime': '0.126099', 'ServiceTime': '0.109810', 'AverageServiceTime': '0.110559', 'ResponseTime': '0.318054', 'InterArrivalTime': '0.762020', 'Throughput': '3.144118', 'AverageThroughput': '4.22549

In [6]:
# hop_df_raw 타입 확인
hop_df_raw.dtypes

Node               object
PerformanceInfo    object
level              object
msg                object
time               object
dtype: object

In [7]:
hop_df = preprocess_hop_log.parse_performance_info(NUM_BROKERS, hop_df_raw)

In [8]:
hop_df

,Node,level,msg,time,B1_Cpu,B1_Memory,B1_QueueLength,B1_QueueTime,B1_ServiceTime,B1_ResponseTime,B1_InterArrivalTime,B1_Throughput,B1_m,B1_TimestampDiff,B2_Cpu,B2_Memory,B2_QueueLength,B2_QueueTime,B2_ServiceTime,B2_ResponseTime,B2_InterArrivalTime,B2_Throughput,B2_m,B2_TimestampDiff,B3_Cpu,B3_Memory,B3_QueueLength,B3_QueueTime,B3_ServiceTime,B3_ResponseTime,B3_InterArrivalTime,B3_Throughput,B3_m,B3_TimestampDiff,B4_Cpu,B4_Memory,B4_QueueLength,B4_QueueTime,B4_ServiceTime,B4_ResponseTime,B4_InterArrivalTime,B4_Throughput,B4_m,B4_TimestampDiff,B5_Cpu,B5_Memory,B5_QueueLength,B5_QueueTime,B5_ServiceTime,B5_ResponseTime,B5_InterArrivalTime,B5_Throughput,B5_m,B5_TimestampDiff
0,localhost:50003,info,Advertisement(apple > 0.25403618120212357) a0938dd9-1e2c-4b30-af19-1da2b3c535b0,2025-01-05 12:13:45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99.809589,23396352.0,0.0,0.104737,0.000000,0.104737,0.000000,9.547724,4.580802,NaN,99.642704,23363584.0,0.0,0.307559,0.000000,0.307559,0.000000,3.251413,4.463063,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,localhost:50003,info,Advertisement(apple > 0.832939761751119) 1d01f9f5-6c5a-4d21-a86b-2c50c72fd32e,2025-01-05 12:13:47,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.007887,23699456.0,0.0,0.101049,0.107739,0.208788,1.020634,4.789538,6.618383,NaN,99.646627,23891968.0,0.0,0.301959,0.314640,0.616599,0.681290,1.621800,6.501333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,localhost:50003,info,Advertisement(apple > 0.3954138949680233) 85c0e4da-8772-4c76-b050-6ff996377687,2025-01-05 12:13:49,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.003245,24051712.0,0.0,0.114638,0.106049,0.220688,1.160354,4.531290,8.071764,NaN,99.685872,24465408.0,1.0,0.300817,0.301745,0.602562,0.608465,1.659579,7.940790,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,localhost:50003,info,Advertisement(apple > 0.40280118482410926) 14bbdb96-e0d9-4e1a-b875-91a242abdb72,2025-01-05 12:13:49,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99.942109,24301568.0,1.0,0.101826,0.118639,0.220466,0.762020,4.535857,8.373761,NaN,99.843099,25096192.0,2.0,0.450594,0.307151,0.757744,0.473828,1.319706,8.256891,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,localhost:50003,info,Advertisement(apple > 0.21107595518065442) 95d3aa9c-e1b6-45a6-a593-e35eeac68db7,2025-01-05 12:13:49,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.544437,24399872.0,0.0,0.208244,0.109810,0.318054,0.762020,3.144118,8.494410,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11817,localhost:50003,info,Advertisement(apple > 0.6055603752201958) 1022799c-ac6a-4eea-8c48-6ee69c62d4f9,2025-01-05 14:13:37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.407501,47845376.0,0.0,0.104185,0.106019,0.210204,0.608553,4.757288,7196.456882,NaN,100.657094,49229824.0,0.0,0.496277,0.215385,0.711661,0.346604,1.405163,7196.339401,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11818,localhost:50003,info,Advertisement(apple > 0.909238792081195) 51d7bb07-c9f3-44a7-8f08-c727c2011da6,2025-01-05 14:13:38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.407358,47845376.0,0.0,0.103631,0.107185,0.210816,0.608609,4.743466,7197.727617,NaN,100.656909,49229824.0,0.0,0.286752,0.207608,0.494360,0.346642,2.022817,7197.609985,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11819,localhost:50003,info,Advertisement(apple > 0.7288474671782913) 163b4b46-56d2-4f33-9139-a73cb1392c56,2025-01-05 14:13:38,100.128889,46125056.0,1.0,0.331219,0.314348,0.645567,0.587131,1.549025,7197.111313,NaN,100.400917,48631808.0,1.0,0.313331,0.316212,0

In [9]:
# 컬럼별 데이터 타입 확인
hop_df.dtypes

Node                    object
level                   object
msg                     object
time                    object
B1_Cpu                 float64
B1_Memory              float64
B1_QueueLength         float64
B1_QueueTime           float64
B1_ServiceTime         float64
B1_ResponseTime        float64
B1_InterArrivalTime    float64
B1_Throughput          float64
B1_m                   float64
B1_TimestampDiff       float64
B2_Cpu                 float64
B2_Memory              float64
B2_QueueLength         float64
B2_QueueTime           float64
B2_ServiceTime         float64
B2_ResponseTime        float64
B2_InterArrivalTime    float64
B2_Throughput          float64
B2_m                   float64
B2_TimestampDiff       float64
B3_Cpu                 float64
B3_Memory              float64
B3_QueueLength         float64
B3_QueueTime           float64
B3_ServiceTime         float64
B3_ResponseTime        float64
B3_InterArrivalTime    float64
B3_Throughput          float64
B3_m    

In [10]:
# hop_df의 time 컬럼의 중복되는 값과 그 개수 확인
# hop_df['time'].value_counts()
hop_df['time'].duplicated().sum()

5449

1초에 2개 이상의 행이 존재할 때\
0을 제외한 값들끼리의 평균값(또는 최댓값)을 구해서 하나의 date에는 하나의 행만 남김

In [11]:
hop_df = aggregate_rows_by_time.handle_grouped_duplicate_time(hop_df, method='mean')


In [12]:
# hop_df의 time 컬럼의 중복값 확인
hop_df['time'].duplicated().sum()

0

In [13]:
# 파일 저장
# 디렉토리가 없으면 생성
if not os.path.exists(path_mergeddf):
    os.makedirs(path_mergeddf)

hop_df.to_csv(path_mergeddf + 'hop_df_agg.csv', index=False)

In [14]:
hop_df

,time,B1_Cpu,B1_Memory,B1_QueueLength,B1_QueueTime,B1_ServiceTime,B1_ResponseTime,B1_InterArrivalTime,B1_Throughput,B1_m,B1_TimestampDiff,B2_Cpu,B2_Memory,B2_QueueLength,B2_QueueTime,B2_ServiceTime,B2_ResponseTime,B2_InterArrivalTime,B2_Throughput,B2_m,B2_TimestampDiff,B3_Cpu,B3_Memory,B3_QueueLength,B3_QueueTime,B3_ServiceTime,B3_ResponseTime,B3_InterArrivalTime,B3_Throughput,B3_m,B3_TimestampDiff,B4_Cpu,B4_Memory,B4_QueueLength,B4_QueueTime,B4_ServiceTime,B4_ResponseTime,B4_InterArrivalTime,B4_Throughput,B4_m,B4_TimestampDiff,B5_Cpu,B5_Memory,B5_QueueLength,B5_QueueTime,B5_ServiceTime,B5_ResponseTime,B5_InterArrivalTime,B5_Throughput,B5_m,B5_TimestampDiff
0,2025-01-05 12:13:45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99.809589,2.339635e+07,0.0,0.104737,0.000000,0.104737,0.000000,9.547724,4.580802,NaN,99.642704,2.336358e+07,0.0,0.307559,0.000000,0.307559,0.000000,3.251413,4.463063,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2025-01-05 12:13:47,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.007887,2.369946e+07,0.0,0.101049,0.107739,0.208788,1.020634,4.789538,6.618383,NaN,99.646627,2.389197e+07,0.0,0.301959,0.314640,0.616599,0.681290,1.621800,6.501333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2025-01-05 12:13:49,95.500628,23732224.0,0.0,0.313170,0.314250,0.627421,1.017234,1.593827,7.878833,NaN,100.286189,24522752.0,2.0,0.433673,0.306935,0.740608,0.652935,1.350243,8.654565,NaN,100.390469,2.461764e+07,0.5,0.139267,0.113155,0.252422,0.733237,4.036870,8.615541,NaN,99.872419,2.506206e+07,2.0,0.560651,0.306833,0.867483,0.501660,1.261448,8.358117,NaN,99.828200,23113728.0,0.0,0.102364,0.100050,0.202413,1.123418,4.940382,7.961741,NaN
3,2025-01-05 12:13:50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.602948,2.544026e+07,0.0,0.113042,0.105800,0.218841,0.524834,4.569526,9.312616,NaN,100.135187,2.580070e+07,3.0,0.714241,0.307194,1.021435,0.396315,0.979015,9.184578,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2025-01-05 12:13:52,96.758355,24420352.0,0.0,0.300166,0.315216,0.615382,0.613816,1.625006,10.952248,NaN,100.297827,25784320.0,0.0,0.491052,0.315814,0.806866,0.613363,1.239364,11.784847,NaN,100.497827,2.597956e+07,0.0,0.111655,0.124005,0.235659,0.642925,4.243531,11.644112,NaN,100.071123,2.739610e+07,5.5,1.610637,0.306365,1.917004,0.330677,0.523093,11.368755,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6368,2025-01-05 14:13:36,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.407571,4.784538e+07,0.0,0.105111,0.106554,0.211665,0.608543,4.724458,7195.129119,NaN,100.657002,4.922982e+07,0.0,0.205853,0.213516,0.419369,0.346621,2.384534,7195.011009,NaN,100.116736,46133248.0,0.0,0.105016,0.104508,0.209524,0.608774,4.772714,7194.822154,NaN
6369,2025-01-05 14:13:37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.407458,4.784538e+07,0.0,0.103602,0.107066,0.210668,0.608569,4.746838,7196.350974,NaN,100.656937,4.922982e+07,0.5,0.451638,0.213502,0.665140,0.346604,1.510833,7196.234224,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6370,2025-01-05 14:13:38,100.128889,46125056.0,1.0,0.331219,0.314348,0.645567,0.587131,1.549025,7197.111313,NaN,100.400917,48631808.0,1.0,0.313331,0.316212,0.629543,0.634876,1.588453,7197.767192,NaN,100.407286,4.784538e+07,0.0,0.103751,0.106909,0.210659,0.608591,4.746999,7197.818340,NaN,100.656909,4.922982e+07,0.0,0.286752,0.207608,0.494360,0.346642,2.022817,7197.609985,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6371,2025-01-05 14:13:39,100.129019,46129152.0,0.0,0.559972,0.307694,0.867667,0.587131,1.152516,7197.429644,NaN,100.400728,48701440.0,0.0,0.618456,0.302252,0.920709,0.634827,1.086120,7198.387650,NaN,100.407232,4.784538e+07,0.0,0.

## Tick Log 전처리

In [15]:
# json 파일을 열고 각 줄을 개별적으로 파싱
# path = '../raw_data/tickLogger/116/'
data = []
with open(path_ticklog + f'{BROKER_PORT}_tick.json', 'r') as file:
    for line in file:
        data.append(json.loads(line))

# 데이터를 DataFrame으로 변환
tick_df_raw = pd.DataFrame(data)
type(tick_df_raw)

pandas.core.frame.DataFrame

In [16]:
# time 컬럼의 중복값과 그 개수 확인

duplicates = tick_df_raw['time'].value_counts()

# 중복이 있는 경우만 출력 (카운트가 1보다 큰 경우)
duplicates = duplicates[duplicates > 1]

print("중복된 time 값과 그 개수:")
print(duplicates)

# 전체 중복 레코드 수 확인
total_duplicates = len(tick_df_raw) - len(tick_df_raw['time'].unique())
print(f"\n총 중복 레코드 수: {total_duplicates}")


중복된 time 값과 그 개수:
2025-01-05 14:09:25    2
2025-01-05 14:12:03    2
2025-01-05 14:03:47    2
2025-01-05 14:11:46    2
2025-01-05 13:59:39    2
                      ..
2025-01-05 14:07:28    2
2025-01-05 13:59:44    2
2025-01-05 14:05:48    2
2025-01-05 14:13:23    2
2025-01-05 13:56:51    2
Name: time, Length: 78, dtype: int64

총 중복 레코드 수: 78


In [17]:
# AverageQueueTime, AverageServiceTime, AverageThroughput, InterArrivalTime, Memory, MessageCount, QueueLength, QueueTime, ResponseTime, ServiceTime, Throughtput, TotalArrival Time 컬럼 값을 float 타입으로 변환
# 여러 컬럼 한 번에 type 변경
tick_df = tick_df_raw.astype({'AverageQueueTime': float, 
                              'AverageServiceTime': float, 
                              'AverageThroughput': float, 
                              'InterArrivalTime': float, 
                              'Memory': float, 
                              'MessageCount': float, 
                              'QueueLength': float, 
                              'QueueTime': float, 
                              'ResponseTime': float, 
                              'ServiceTime': float, 
                              'Throughput': float, 
                              'TotalArrival Time': float})


In [18]:
# 파일 저장
# 디렉토리가 없으면 생성
if not os.path.exists(path_mergeddf):
    os.makedirs(path_mergeddf)

tick_df.to_csv(path_mergeddf + 'tick_df.csv', index=False)

In [19]:
tick_df.dtypes


AverageQueueTime      float64
AverageServiceTime    float64
AverageThroughput     float64
Bottleneck               bool
Cpu                   float64
InterArrivalTime      float64
Memory                float64
MessageCount          float64
QueueLength           float64
QueueTime             float64
ResponseTime          float64
ServiceTime           float64
Throughput            float64
TotalArrival Time     float64
level                  object
msg                    object
time                   object
dtype: object

In [20]:
tick_df

,AverageQueueTime,AverageServiceTime,AverageThroughput,Bottleneck,Cpu,InterArrivalTime,Memory,MessageCount,QueueLength,QueueTime,ResponseTime,ServiceTime,Throughput,TotalArrival Time,level,msg,time
0,0.000000,0.000000,0.000000,False,102.310488,0.000000,21368832.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,info,Performance Metrics,2025-01-05 12:13:41
1,0.000000,0.000000,0.000000,False,100.396299,0.000000,23146496.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,info,Performance Metrics,2025-01-05 12:13:42
2,0.000000,0.000000,0.000000,False,100.423737,0.000000,23158784.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,info,Performance Metrics,2025-01-05 12:13:43
3,0.000000,0.000000,0.000000,False,100.198075,0.000000,23162880.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,info,Performance Metrics,2025-01-05 12:13:44
4,0.104737,0.107739,4.706414,False,100.094002,0.000000,23572480.0,1.0,0.0,0.104737,0.212476,0.107739,4.706414,0.000000,info,Performance Metrics,2025-01-05 12:13:45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7194,0.106688,0.107096,4.677626,False,100.407546,0.608527,47845376.0,11815.0,0.0,0.103553,0.210107,0.106554,4.759486,7189.749086,info,Performance Metrics,2025-01-05 14:13:35
7195,0.106688,0.107096,4.677627,False,100.407528,0.608543,47845376.0,11816.0,0.0,0.105111,0.213223,0.108112,4.689928,7190.547943,info,Performance Metrics,2025-01-05 14:13:36
7196,0.106687,0.107096,4.677640,False,100.407306,0.608553,47845376.0,11818.0,0.0,0.104185,0.211370,0.107185,4.731049,7191.876632,info,Performance Metrics,2025-01-05 14:13:38
7197,0.106687,0.107096,4.677654,False,100.407258,0.608573,47845376.0,11820.0,0.0,0.103871,0.209769,0.105898,4.767153,7193.329126,info,Performance Metrics,2025-01-05 14:13:39


In [21]:
merged_df = merge_hop_tick.merge_dataframes(hop_df, tick_df, BROKER_PORT)

In [22]:
merged_df

,AverageQueueTime,AverageServiceTime,AverageThroughput,Bottleneck,B3_Cpu,B3_InterArrivalTime,B3_Memory,MessageCount,B3_QueueLength,B3_QueueTime,B3_ResponseTime,B3_ServiceTime,B3_Throughput,TotalArrival Time,level,msg,time,B1_Cpu,B1_Memory,B1_QueueLength,B1_QueueTime,B1_ServiceTime,B1_ResponseTime,B1_InterArrivalTime,B1_Throughput,B1_m,B1_TimestampDiff,B2_Cpu,B2_Memory,B2_QueueLength,B2_QueueTime,B2_ServiceTime,B2_ResponseTime,B2_InterArrivalTime,B2_Throughput,B2_m,B2_TimestampDiff,B3_m,B3_TimestampDiff,B4_Cpu,B4_Memory,B4_QueueLength,B4_QueueTime,B4_ServiceTime,B4_ResponseTime,B4_InterArrivalTime,B4_Throughput,B4_m,B4_TimestampDiff,B5_Cpu,B5_Memory,B5_QueueLength,B5_QueueTime,B5_ServiceTime,B5_ResponseTime,B5_InterArrivalTime,B5_Throughput,B5_m,B5_TimestampDiff
0,0.000000,0.000000,0.000000,False,102.310488,0.000000,21368832.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,info,Performance Metrics,2025-01-05 12:13:41,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.000000,0.000000,0.000000,False,100.396299,0.000000,23146496.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,info,Performance Metrics,2025-01-05 12:13:42,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.000000,0.000000,0.000000,False,100.423737,0.000000,23158784.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,info,Performance Metrics,2025-01-05 12:13:43,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.000000,0.000000,0.000000,False,100.198075,0.000000,23162880.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,info,Performance Metrics,2025-01-05 12:13:44,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.104737,0.107739,4.706414,False,99.809589,0.000000,23396352.0,1.0,0.0,0.104737,0.104737,0.000000,9.547724,0.000000,info,Performance Metrics,2025-01-05 12:13:45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.580802,NaN,99.642704,23363584.0,0.0,0.307559,0.000000,0.307559,0.000000,3.251413,4.463063,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7194,0.106688,0.107096,4.677626,False,100.407280,0.608542,47861760.0,11815.0,0.0,0.102044,0.206822,0.104778,4.835668,7189.749086,info,Performance Metrics,2025-01-05 14:13:35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7194.192917,NaN,100.656831,49227776.0,0.0,0.209410,0.213428,0.422839,0.346601,2.365019,7194.076374,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7195,0.106688,0.107096,4.677627,False,100.407571,0.608543,47845376.0,11816.0,0.0,0.105111,0.211665,0.106554,4.724458,7190.547943,info,Performance Metrics,2025-01-05 14:13:36,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7195.129119,NaN,100.657002,49229824.0,0.0,0.205853,0.213516,0.419369,0.346621,2.384534,7195.011009,NaN,100.116736,46133248.0,0.0,0.105016,0.104508,0.209524,0.608774,4.772714,7194.822154,NaN
7196,0.106687,0.107096,4.677640,False,100.407286,0.608591,47845376.0,11818.0,0.0,0.103751,0.210659,0.106909,4.746999,7191.876632,info,Performance Metrics,2025-01-05 14:13:38,100.128889,46125056.0,1.0,0.331219,0.314348,0.645567,0.587131,1.549025,7197.111313,NaN,100.400917,48631808.0,1.0,0.313331,0.316212,0.629543,0.634876,1.588453,7197.767192,NaN,7197.818340,NaN,100.656909,49229824.0,0.0,0.286752,0.207608,0.494360,0.346642,2.022817,7197.

In [23]:
merged_df['Bottleneck'] = merged_df['Bottleneck'].astype('float64')
merged_df.columns

Index(['AverageQueueTime', 'AverageServiceTime', 'AverageThroughput',
       'Bottleneck', 'B3_Cpu', 'B3_InterArrivalTime', 'B3_Memory',
       'MessageCount', 'B3_QueueLength', 'B3_QueueTime', 'B3_ResponseTime',
       'B3_ServiceTime', 'B3_Throughput', 'TotalArrival Time', 'level', 'msg',
       'time', 'B1_Cpu', 'B1_Memory', 'B1_QueueLength', 'B1_QueueTime',
       'B1_ServiceTime', 'B1_ResponseTime', 'B1_InterArrivalTime',
       'B1_Throughput', 'B1_m', 'B1_TimestampDiff', 'B2_Cpu', 'B2_Memory',
       'B2_QueueLength', 'B2_QueueTime', 'B2_ServiceTime', 'B2_ResponseTime',
       'B2_InterArrivalTime', 'B2_Throughput', 'B2_m', 'B2_TimestampDiff',
       'B3_m', 'B3_TimestampDiff', 'B4_Cpu', 'B4_Memory', 'B4_QueueLength',
       'B4_QueueTime', 'B4_ServiceTime', 'B4_ResponseTime',
       'B4_InterArrivalTime', 'B4_Throughput', 'B4_m', 'B4_TimestampDiff',
       'B5_Cpu', 'B5_Memory', 'B5_QueueLength', 'B5_QueueTime',
       'B5_ServiceTime', 'B5_ResponseTime', 'B5_InterArrivalTime',

In [24]:
# 학습에 불필요한 컬럼 제거
# 불필요한 컬럼 리스트
drop_columns = [
    'level',
    'msg',
    # 'Node',
    # 'TotalArrivalTime',
    'AverageQueueTime',
    'AverageServiceTime',
    'AverageThroughput',
    
    'MessageCount',
    # 'Timestamp'
]

# 불필요한 컬럼 제거
merged_df.drop(columns=drop_columns, inplace=True)

In [25]:
merged_df = merged_df.rename(columns={'time': 'date'})

In [26]:
merged_df

,Bottleneck,B3_Cpu,B3_InterArrivalTime,B3_Memory,B3_QueueLength,B3_QueueTime,B3_ResponseTime,B3_ServiceTime,B3_Throughput,TotalArrival Time,date,B1_Cpu,B1_Memory,B1_QueueLength,B1_QueueTime,B1_ServiceTime,B1_ResponseTime,B1_InterArrivalTime,B1_Throughput,B1_m,B1_TimestampDiff,B2_Cpu,B2_Memory,B2_QueueLength,B2_QueueTime,B2_ServiceTime,B2_ResponseTime,B2_InterArrivalTime,B2_Throughput,B2_m,B2_TimestampDiff,B3_m,B3_TimestampDiff,B4_Cpu,B4_Memory,B4_QueueLength,B4_QueueTime,B4_ServiceTime,B4_ResponseTime,B4_InterArrivalTime,B4_Throughput,B4_m,B4_TimestampDiff,B5_Cpu,B5_Memory,B5_QueueLength,B5_QueueTime,B5_ServiceTime,B5_ResponseTime,B5_InterArrivalTime,B5_Throughput,B5_m,B5_TimestampDiff
0,0.0,102.310488,0.000000,21368832.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,2025-01-05 12:13:41,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,100.396299,0.000000,23146496.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,2025-01-05 12:13:42,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.0,100.423737,0.000000,23158784.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,2025-01-05 12:13:43,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.0,100.198075,0.000000,23162880.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,2025-01-05 12:13:44,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.0,99.809589,0.000000,23396352.0,0.0,0.104737,0.104737,0.000000,9.547724,0.000000,2025-01-05 12:13:45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.580802,NaN,99.642704,23363584.0,0.0,0.307559,0.000000,0.307559,0.000000,3.251413,4.463063,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7194,0.0,100.407280,0.608542,47861760.0,0.0,0.102044,0.206822,0.104778,4.835668,7189.749086,2025-01-05 14:13:35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7194.192917,NaN,100.656831,49227776.0,0.0,0.209410,0.213428,0.422839,0.346601,2.365019,7194.076374,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7195,0.0,100.407571,0.608543,47845376.0,0.0,0.105111,0.211665,0.106554,4.724458,7190.547943,2025-01-05 14:13:36,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7195.129119,NaN,100.657002,49229824.0,0.0,0.205853,0.213516,0.419369,0.346621,2.384534,7195.011009,NaN,100.116736,46133248.0,0.0,0.105016,0.104508,0.209524,0.608774,4.772714,7194.822154,NaN
7196,0.0,100.407286,0.608591,47845376.0,0.0,0.103751,0.210659,0.106909,4.746999,7191.876632,2025-01-05 14:13:38,100.128889,46125056.0,1.0,0.331219,0.314348,0.645567,0.587131,1.549025,7197.111313,NaN,100.400917,48631808.0,1.0,0.313331,0.316212,0.629543,0.634876,1.588453,7197.767192,NaN,7197.818340,NaN,100.656909,49229824.0,0.0,0.286752,0.207608,0.494360,0.346642,2.022817,7197.609985,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7197,0.0,100.407232,0.608574,47845376.0,0.0,0.104643,0.210541,0.105898,4.749673,7193.329126,2025-01-05 14:13:39,100.129019,46129152.0,0.0,0.559972,0.307694,0.867667,0.587131,1.152516,7197.429644,NaN,100.400728,48701440.0,0.0,0.618456,0.302252,0.920709,0.634827,1.086120,7198.387650,NaN,7198.530291,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
# # 모든 값이 NA인 열들을 확인 후 DataFrame에서 삭제
# print(merged_df.columns[merged_df.isna().all()].tolist())
# merged_df.drop(columns=merged_df.columns[merged_df.isna().all()].tolist(), inplace=True)


In [28]:
# merge_df 의 컬럼 출력
merged_df.columns

Index(['Bottleneck', 'B3_Cpu', 'B3_InterArrivalTime', 'B3_Memory',
       'B3_QueueLength', 'B3_QueueTime', 'B3_ResponseTime', 'B3_ServiceTime',
       'B3_Throughput', 'TotalArrival Time', 'date', 'B1_Cpu', 'B1_Memory',
       'B1_QueueLength', 'B1_QueueTime', 'B1_ServiceTime', 'B1_ResponseTime',
       'B1_InterArrivalTime', 'B1_Throughput', 'B1_m', 'B1_TimestampDiff',
       'B2_Cpu', 'B2_Memory', 'B2_QueueLength', 'B2_QueueTime',
       'B2_ServiceTime', 'B2_ResponseTime', 'B2_InterArrivalTime',
       'B2_Throughput', 'B2_m', 'B2_TimestampDiff', 'B3_m', 'B3_TimestampDiff',
       'B4_Cpu', 'B4_Memory', 'B4_QueueLength', 'B4_QueueTime',
       'B4_ServiceTime', 'B4_ResponseTime', 'B4_InterArrivalTime',
       'B4_Throughput', 'B4_m', 'B4_TimestampDiff', 'B5_Cpu', 'B5_Memory',
       'B5_QueueLength', 'B5_QueueTime', 'B5_ServiceTime', 'B5_ResponseTime',
       'B5_InterArrivalTime', 'B5_Throughput', 'B5_m', 'B5_TimestampDiff'],
      dtype='object')

In [29]:
new_order = ['date', 'Bottleneck',
            #  'HopCount', 
            #  'B1_Timestamp', 'B1_BrokerId',
             'B1_Cpu', 'B1_Memory', 'B1_InterArrivalTime', 'B1_QueueLength', 'B1_QueueTime', 'B1_ServiceTime', 'B1_ResponseTime', 'B1_Throughput',
            #  'B2_Timestamp',  'B2_BrokerId',
             'B2_Cpu', 'B2_Memory', 'B2_InterArrivalTime', 'B2_QueueLength',  'B2_QueueTime', 'B2_ServiceTime', 'B2_ResponseTime', 'B2_Throughput',
            #  'B3_Timestamp', 'B3_BrokerId',
             'B3_Cpu', 'B3_Memory', 'B3_InterArrivalTime', 'B3_QueueLength', 'B3_QueueTime', 'B3_ServiceTime', 'B3_ResponseTime', 'B3_Throughput',
            #  'B4_Timestamp', 'B4_BrokerId',
             'B4_Cpu', 'B4_Memory', 'B4_InterArrivalTime', 'B4_QueueLength', 'B4_QueueTime', 'B4_ServiceTime', 'B4_ResponseTime', 'B4_Throughput',
            #  'B5_Timestamp', 'B5_BrokerId',
             'B5_Cpu', 'B5_Memory', 'B5_InterArrivalTime', 'B5_QueueLength', 'B5_QueueTime', 'B5_ServiceTime', 'B5_ResponseTime', 'B5_Throughput',]

# 새로운 순서로 데이터프레임을 재정렬합니다
merged_df = merged_df[new_order]

In [30]:
drop_columns = [ 
    'Bottleneck',
#     'BrokerId_0',
#     'BrokerId_1',
#     'BrokerId_2'
]

# # 불필요한 컬럼 제거
merged_df.drop(columns=drop_columns, inplace=True)

c:\Users\SJSJ\anaconda3\envs\informer2020\lib\site-packages\pandas\core\frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [31]:
# return all unique values of merged_df columns
for col in merged_df.columns:
    print(col)
    print(merged_df[col].unique())
    print()


date
['2025-01-05 12:13:41' '2025-01-05 12:13:42' '2025-01-05 12:13:43' ...
 '2025-01-05 14:13:36' '2025-01-05 14:13:38' '2025-01-05 14:13:39']

B1_Cpu
[        nan  95.500628   96.758355   97.541092   97.537551   98.704301
  98.7321     99.246406   99.463315   99.502735   99.571371   99.582123
  99.598643   99.676712   99.70832    99.718348   99.750116   99.777493
  99.783238   99.794565   99.790002   99.7714     99.798289   99.883151
  99.903604   99.904103   99.905136   99.90648    99.919967   99.933496
  99.937979   99.939501   99.94349    99.944073   99.948212   99.951208
  99.957584   99.955381   99.964405   99.963058   99.963488   99.971699
  99.974467   99.979813   99.984379   99.985847   99.994064   99.992421
  99.998638  100.002103  100.002554  100.000296  100.002083  100.003005
  99.996346  100.00683   100.005003  100.005102  100.003517  100.00974
 100.013678  100.01415   100.00129   100.000797  100.00199   100.010975
 100.008008  100.0143    100.028109  100.026771  100.0311

In [32]:
# NaN 값을 0으로 대체
merged_df.fillna(0, inplace=True)

c:\Users\SJSJ\anaconda3\envs\informer2020\lib\site-packages\pandas\core\frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [33]:
merged_df

,date,B1_Cpu,B1_Memory,B1_InterArrivalTime,B1_QueueLength,B1_QueueTime,B1_ServiceTime,B1_ResponseTime,B1_Throughput,B2_Cpu,B2_Memory,B2_InterArrivalTime,B2_QueueLength,B2_QueueTime,B2_ServiceTime,B2_ResponseTime,B2_Throughput,B3_Cpu,B3_Memory,B3_InterArrivalTime,B3_QueueLength,B3_QueueTime,B3_ServiceTime,B3_ResponseTime,B3_Throughput,B4_Cpu,B4_Memory,B4_InterArrivalTime,B4_QueueLength,B4_QueueTime,B4_ServiceTime,B4_ResponseTime,B4_Throughput,B5_Cpu,B5_Memory,B5_InterArrivalTime,B5_QueueLength,B5_QueueTime,B5_ServiceTime,B5_ResponseTime,B5_Throughput
0,2025-01-05 12:13:41,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,102.310488,21368832.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
1,2025-01-05 12:13:42,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,100.396299,23146496.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
2,2025-01-05 12:13:43,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,100.423737,23158784.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
3,2025-01-05 12:13:44,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,100.198075,23162880.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
4,2025-01-05 12:13:45,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,99.809589,23396352.0,0.000000,0.0,0.104737,0.000000,0.104737,9.547724,99.642704,23363584.0,0.000000,0.0,0.307559,0.000000,0.307559,3.251413,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7194,2025-01-05 14:13:35,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,100.407280,47861760.0,0.608542,0.0,0.102044,0.104778,0.206822,4.835668,100.656831,49227776.0,0.346601,0.0,0.209410,0.213428,0.422839,2.365019,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
7195,2025-01-05 14:13:36,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,100.407571,47845376.0,0.608543,0.0,0.105111,0.106554,0.211665,4.724458,100.657002,49229824.0,0.346621,0.0,0.205853,0.213516,0.419369,2.384534,100.116736,46133248.0,0.608774,0.0,0.105016,0.104508,0.209524,4.772714
7196,2025-01-05 14:13:38,100.128889,46125056.0,0.587131,1.0,0.331219,0.314348,0.645567,1.549025,100.400917,48631808.0,0.634876,1.0,0.313331,0.316212,0.629543,1.588453,100.407286,47845376.0,0.608591,0.0,0.103751,0.106909,0.210659,4.746999,100.656909,49229824.0,0.346642,0.0,0.286752,0.207608,0.494360,2.022817,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
7197,2025-01-05 14:13:39,100.129019,46129152.0,0.587131,0.0,0.559972,0.307694,0.867667,1.152516,100.400728,48701440.0,0.634827,0.0,0.618456,0.302252,0.920709,1.086120,100.407232,47845376.0,0.608574,0.0,0.104643,0.105898,0.210541,4.749673,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000


In [34]:
# # 각 컬럼별로 unique 값이 하나인 컬럼 삭제
# for col in merged_df.columns:
#     if len(merged_df[col].unique()) == 1:
#         print(col)
#         merged_df = merged_df.drop(col, axis=1)

In [35]:
# save to csv
# 디렉토리가 없으면 생성
if not os.path.exists(path_mergeddf):
    os.makedirs(path_mergeddf)

merged_df.to_csv(path_mergeddf +'merged_df.csv', date_format='%Y-%m-%d %H:%M:%S', index=False)


In [36]:
merged_df.dtypes

date                    object
B1_Cpu                 float64
B1_Memory              float64
B1_InterArrivalTime    float64
B1_QueueLength         float64
B1_QueueTime           float64
B1_ServiceTime         float64
B1_ResponseTime        float64
B1_Throughput          float64
B2_Cpu                 float64
B2_Memory              float64
B2_InterArrivalTime    float64
B2_QueueLength         float64
B2_QueueTime           float64
B2_ServiceTime         float64
B2_ResponseTime        float64
B2_Throughput          float64
B3_Cpu                 float64
B3_Memory              float64
B3_InterArrivalTime    float64
B3_QueueLength         float64
B3_QueueTime           float64
B3_ServiceTime         float64
B3_ResponseTime        float64
B3_Throughput          float64
B4_Cpu                 float64
B4_Memory              float64
B4_InterArrivalTime    float64
B4_QueueLength         float64
B4_QueueTime           float64
B4_ServiceTime         float64
B4_ResponseTime        float64
B4_Throu

In [37]:
merged_df.columns

Index(['date', 'B1_Cpu', 'B1_Memory', 'B1_InterArrivalTime', 'B1_QueueLength',
       'B1_QueueTime', 'B1_ServiceTime', 'B1_ResponseTime', 'B1_Throughput',
       'B2_Cpu', 'B2_Memory', 'B2_InterArrivalTime', 'B2_QueueLength',
       'B2_QueueTime', 'B2_ServiceTime', 'B2_ResponseTime', 'B2_Throughput',
       'B3_Cpu', 'B3_Memory', 'B3_InterArrivalTime', 'B3_QueueLength',
       'B3_QueueTime', 'B3_ServiceTime', 'B3_ResponseTime', 'B3_Throughput',
       'B4_Cpu', 'B4_Memory', 'B4_InterArrivalTime', 'B4_QueueLength',
       'B4_QueueTime', 'B4_ServiceTime', 'B4_ResponseTime', 'B4_Throughput',
       'B5_Cpu', 'B5_Memory', 'B5_InterArrivalTime', 'B5_QueueLength',
       'B5_QueueTime', 'B5_ServiceTime', 'B5_ResponseTime', 'B5_Throughput'],
      dtype='object')

In [38]:
# date 컬럼의 중복값과 그 개수 확인

duplicates = merged_df['date'].value_counts()

# 중복이 있는 경우만 출력 (카운트가 1보다 큰 경우)
duplicates = duplicates[duplicates > 1]

print("중복된 date 값과 그 개수:")
print(duplicates)

# 전체 중복 레코드 수 확인
total_duplicates = len(merged_df) - len(merged_df['date'].unique())
print(f"\n총 중복 레코드 수: {total_duplicates}")


중복된 date 값과 그 개수:
2025-01-05 14:09:25    2
2025-01-05 14:12:03    2
2025-01-05 14:03:47    2
2025-01-05 14:11:46    2
2025-01-05 13:59:39    2
                      ..
2025-01-05 14:07:28    2
2025-01-05 13:59:44    2
2025-01-05 14:05:48    2
2025-01-05 14:13:23    2
2025-01-05 13:56:51    2
Name: date, Length: 78, dtype: int64

총 중복 레코드 수: 78


---

## Temporal-Spatio dataset 구축

In [39]:
# # 사용할 feature 선택

# # 해당 리스트에 있는 문자열로 끝나는 컬럼만 선택
# feature_list = ['Cpu', 'Memory', 'InterArrivalTime', 'QueueLength', 'QueueTime', 'ServiceTime', 'ResponseTime', 'Throughput']

# # feature_list에 있는 문자열로 끝나는 컬럼만 선택
# selected_features = merged_df.filter(regex='|'.join(feature_list))

# selected_features
